In [1]:
from math import log, exp
import pandas as pd
import nltk
from n_gram import *


In [2]:
filepath = "Dataset/Training/train_data.csv"
df = pd.read_csv(filepath)
df.head()

,0
0,seriously ask any woman they will tell you thi...
1,in us of taxpayers paid tax
2,haha russia go boom splat india go ahhh plop
3,can t say i m surprised
4,they use the religious label for social contro...


In [4]:
processed_corpus = {} 
sentence_id = 0
for sentence in df['Value']:
    tokenized_sentence = []
    tokens = nltk.word_tokenize(sentence)
    tokens = (token for token in tokens if token.isalpha())
    tokenized_sentence += tokens
    processed_corpus[sentence_id] = tokenized_sentence
    sentence_id += 1


In [5]:
print(processed_corpus[0])

['seriously', 'ask', 'any', 'woman', 'they', 'will', 'tell', 'you', 'this', 'is', 'all', 'very', 'common']


Unigram Model

In [6]:
unigram_counts = {}
unigram_counts["<s>"] = 0
unigram_counts["</s>"] = 0
for id in processed_corpus:
    for token in processed_corpus[id]:
        if token not in unigram_counts:
            unigram_counts[token] = 0
        unigram_counts[token] += 1
    unigram_counts["<s>"] += 1
    unigram_counts["</s>"] += 1


In [7]:
unigram_counts

{'<s>': 263000,
 '</s>': 263000,
 'seriously': 615,
 'ask': 1038,
 'any': 7706,
 'woman': 602,
 'they': 31081,
 'will': 11433,
 'tell': 1568,
 'you': 56610,
 'this': 33315,
 'is': 61510,
 'all': 14103,
 'very': 4379,
 'common': 931,
 'in': 57995,
 'us': 8300,
 'of': 70807,
 'taxpayers': 20,
 'paid': 519,
 'tax': 533,
 'haha': 414,
 'russia': 651,
 'go': 4428,
 'boom': 45,
 'splat': 1,
 'india': 7464,
 'ahhh': 36,
 'plop': 1,
 'can': 16704,
 't': 31382,
 'say': 4934,
 'i': 62232,
 'm': 6714,
 'surprised': 395,
 'use': 2358,
 'the': 150890,
 'religious': 1641,
 'label': 112,
 'for': 32610,
 'social': 1305,
 'control': 975,
 'and': 83781,
 'manipulation': 70,
 'people': 21335,
 'yea': 248,
 'used': 2173,
 'to': 93867,
 'do': 13415,
 'that': 50836,
 'too': 4732,
 'but': 23334,
 'then': 6128,
 'higher': 532,
 'classes': 230,
 'homework': 34,
 'given': 1024,
 'kept': 267,
 'on': 21367,
 'growing': 213,
 'it': 52240,
 'just': 15354,
 'felt': 304,
 'as': 20063,
 'a': 73271,
 'chore': 4,
 'top'

In [8]:
k = train_n_gram(processed_corpus, 1)
k

{'<s>': 0.06164710757865632,
 'seriously': 0.0001441557838816488,
 'ask': 0.00024330683523439263,
 'any': 0.001806283692019489,
 'woman': 0.0001411085884500042,
 'they': 0.007285375477765084,
 'will': 0.0026798911823071397,
 'tell': 0.00036753864898605743,
 'you': 0.013269364106569332,
 'this': 0.0078090242927107804,
 'is': 0.01441792238465076,
 'all': 0.0033057382440372244,
 'very': 0.001026436061167057,
 'common': 0.00021822607283547162,
 '</s>': 0.06164710757865632,
 'in': 0.013594007619863777,
 'us': 0.0019455170832807886,
 'of': 0.016597135917573832,
 'taxpayers': 4.687992971760937e-06,
 'paid': 0.00012165341761719631,
 'tax': 0.00012493501269742897,
 'haha': 9.70414545154514e-05,
 'russia': 0.0001525941712308185,
 'go': 0.0010379216439478714,
 'boom': 1.0547984186462107e-05,
 'splat': 2.3439964858804683e-07,
 'india': 0.0017495589770611815,
 'ahhh': 8.438387349169686e-06,
 'plop': 2.3439964858804683e-07,
 'can': 0.003915411730014734,
 't': 0.007355929771990086,
 'say': 0.00115652

In [ ]:
len(unigram_counts)

66112

In [9]:
filepath = "Dataset/Testing/test_data.csv"
df_test = pd.read_csv(filepath)
df_test.head()

,0
0,actually that s a really amazing idea mashaallah
1,the chinese fella was trying to take him on li...
2,i honestly think international troops will be ...
3,marry man s best friend if the man isn t good ...
4,if climate change will submerge all those plac...


In [ ]:
test_corpus = {}
test_tokens = set()
sentence_id = 0
num_test_words = 0
for sentence in df_test['Value']:
    tokenized_sentence = []
    tokens = nltk.word_tokenize(sentence)
    tokens = (token for token in tokens if token.isalpha())
    tokenized_sentence += tokens
    test_corpus[sentence_id] = tokenized_sentence
    num_test_words += 2 + len(tokenized_sentence)
    for token in tokenized_sentence:
        test_tokens.add(token)
    sentence_id += 1

In [ ]:
N_train = 0
for token in unigram_counts:
    N_train += unigram_counts[token]

In [ ]:
len(test_tokens)

35292

In [ ]:
# calculating perplexity

prob_uni_words = {}
prob_uni_words["<s>"] = unigram_counts["<s>"] / N_train
prob_uni_words["</s>"] = unigram_counts["</s>"] / N_train
for token in test_tokens:
        if token in unigram_counts:
            prob_uni_words[token] = unigram_counts[token]/N_train
        else:
            prob_uni_words[token] = 0

In [ ]:
out_of_vocab = 0
for i in prob_uni_words:
    if(prob_uni_words[i]==0):
        out_of_vocab+=1
out_of_vocab

6865

In [ ]:
perplexity = {}
ep=1e-15
pp = 0
for id in test_corpus:
    N = 2 + len(test_corpus[id])
    temp = log(prob_uni_words["<s>"])
    for token in test_corpus[id]:
        if(prob_uni_words[token]==0):
            temp += log(prob_uni_words[token]+ep)
        else:
            temp += log(prob_uni_words[token])
    temp += log(prob_uni_words["</s>"])
    temp = (-temp)/N
    perplexity[id] = exp(temp)
    pp += perplexity[id]


In [ ]:
Unigram_pp = pp/len(test_corpus)
print("Perplexity of Unigram Model without smoothing is: ", Unigram_pp)

Perplexity of Unigram Model without smoothing is:  255029160.07513976


Unigram Model with Laplace Smoothing

In [ ]:
vocab = set()
for id in processed_corpus:
    for token in processed_corpus[id]:
        vocab.add(token)
print(len(vocab))
for id in test_corpus:
    for token in test_corpus[id]:
        vocab.add(token)
print(len(vocab))
V = len(vocab) + 2

66110
72975


In [ ]:
prob_uni_sm_words = {}
prob_uni_sm_words["<s>"] = (unigram_counts["<s>"] + 1) / (N_train + V)
prob_uni_sm_words["</s>"] = (unigram_counts["</s>"] + 1) / (N_train + V)
for token in test_tokens:
        if token in unigram_counts:
            prob_uni_sm_words[token] = (unigram_counts[token] + 1) / (N_train + V)
        else:
              prob_uni_sm_words[token] = 1 / (N_train + V)

In [ ]:
perplexity_sm = {}
pp_sm = 0
for id in test_corpus:
    N = 2 + len(test_corpus[id])
    temp = log(prob_uni_sm_words["<s>"])
    for token in test_corpus[id]:
        temp += log(prob_uni_sm_words[token])
    temp += log(prob_uni_sm_words["</s>"])
    temp=(-temp)/N
    perplexity_sm[id] = exp(temp)
    pp_sm += perplexity_sm[id]

In [ ]:
Unigram_sm_pp = pp_sm/len(test_corpus)
print("Perplexity of Unigram Model with Laplace Smoothing is: ", Unigram_sm_pp)

Perplexity of Unigram Model with Laplace Smoothing is:  1277.93160029543


Bigram Model

In [13]:
bigram_counts = {}
for id in processed_corpus:
    tokens = processed_corpus[id]
    size = len(tokens)
    if(("<s> "+tokens[0]) not in bigram_counts):
        bigram_counts["<s> "+ tokens[0]] = 0
    bigram_counts["<s> "+tokens[0]] += 1
    for i in range(0,size-1):
        str = tokens[i] + " " + tokens[i+1]
        if(str not in bigram_counts):
            bigram_counts[str] = 0
        bigram_counts[str] += 1
    if ((tokens[size-1] + " </s>") not in bigram_counts):
        bigram_counts[tokens[size-1] + " </s>"] = 0
    bigram_counts[tokens[size-1] + " </s>"]+= 1
    

In [ ]:
train_n_gram(processed_corpus, 2)


{'<s> seriously': 133,
 'seriously ask': 2,
 'ask any': 29,
 'any woman': 5,
 'woman they': 2,
 'they will': 904,
 'will tell': 58,
 'tell you': 304,
 'you this': 58,
 'this is': 5756,
 'is all': 255,
 'all very': 22,
 'very common': 40,
 'common </s>': 49,
 '<s> in': 2234,
 'in us': 173,
 'us of': 9,
 'of taxpayers': 2,
 'taxpayers paid': 1,
 'paid tax': 2,
 'tax </s>': 49,
 '<s> haha': 265,
 'haha russia': 1,
 'russia go': 1,
 'go boom': 1,
 'boom splat': 1,
 'splat india': 1,
 'india go': 3,
 'go ahhh': 1,
 'ahhh plop': 1,
 'plop </s>': 1,
 '<s> can': 1047,
 'can t': 3217,
 't say': 280,
 'say i': 161,
 'i m': 6353,
 'm surprised': 54,
 'surprised </s>': 36,
 '<s> they': 5781,
 'they use': 110,
 'use the': 209,
 'the religious': 80,
 'religious label': 1,
 'label for': 1,
 'for social': 20,
 'social control': 1,
 'control and': 58,
 'and manipulation': 7,
 'manipulation of': 5,
 'of the': 12759,
 'the people': 2110,
 'people </s>': 1801,
 '<s> yea': 174,
 'yea i': 28,
 'i used': 248

In [ ]:
len(bigram_counts)

871874

In [ ]:
prob_bi_words = {}
for id in test_corpus:
    tokens = ["<s>"]
    tokens += test_corpus[id]
    tokens.append("</s>")
    size = len(tokens)
    for i in range(0,size-1):
        key = tokens[i]+ " "+ tokens[i+1]
        if key in bigram_counts:
            prob_bi_words[key] = bigram_counts[key] / unigram_counts[tokens[i]]
        else:
            prob_bi_words[key] = 0

    

In [ ]:
bi_perplexity = {}
bi_pp = 0
ep=1e-15
for id in test_corpus:
    N = 2 + len(test_corpus[id])
    tokens = ["<s>"]
    tokens += test_corpus[id]
    tokens.append("</s>")
    size = len(tokens)
    temp = 0
    for i in range(0,size-1):
        str = tokens[i] + " " + tokens[i+1]
        if prob_bi_words[str]!=0:
            temp += log(prob_bi_words[str])
        else:
            temp += log(prob_bi_words[str]+ep)
    temp = (-temp)/N
    bi_perplexity[id] = exp(temp)
    bi_pp += bi_perplexity[id]

In [ ]:
Bigram_pp = bi_pp/len(test_corpus)
print("Perplexity of Bigram Model without smoothing is: ", Bigram_pp)

Perplexity of Bigram Model without smoothing is:  31069869118.23472


Bigram Model with Laplace Smoothing

In [ ]:
prob_bi_sm_words = {}
for key in bigram_counts:
    w1,w2 = key.split(" ")
    if(key not in prob_bi_sm_words):
        prob_bi_sm_words[key] = (bigram_counts[key] + 1) / (unigram_counts[w1] + V)

In [ ]:
bi_perplexity_sm = {}
bi_pp_sm = 0
for id in test_corpus:
    N = 2 + len(test_corpus[id])
    tokens = ["<s>"]
    tokens += test_corpus[id]
    tokens.append("</s>")
    size = len(tokens)
    temp = 0
    for i in range(0,size-1):
        str = tokens[i] + " " + tokens[i+1]
        if(str in prob_bi_sm_words):
            temp += log(prob_bi_sm_words[str])
        elif (tokens[i] in unigram_counts):
            p = 1 / (unigram_counts[tokens[i]] + V)
            temp += log(p)
        else:
            p = 1/V
            temp += log(p)
    temp = (-temp)/N
    bi_perplexity_sm[id] = exp(temp)
    bi_pp_sm += bi_perplexity_sm[id]

In [ ]:
Bigram_sm_pp = bi_pp_sm/len(test_corpus)
print("Perplexity of Bigram Model with smoothing is: ", Bigram_sm_pp)

Perplexity of Bigram Model with smoothing is:  1985.1815331914747


Trigram Model

In [10]:
trigram_counts = {}
for id in processed_corpus:
    tokens = ["<s>", "<s>"]
    tokens += processed_corpus[id] + ["</s>", "</s>"]
    size = len(tokens)
    for i in range(0,size-2):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
        if(str not in trigram_counts):
            trigram_counts[str] = 0
        trigram_counts[str] += 1

In [11]:
print(len(trigram_counts))

2136765


In [14]:
prob_tri_words = {}
for key in trigram_counts:
    w1,w2,w3 = key.split(" ")
    prev = w1 + " " + w2
    if (prev in bigram_counts):
        prob_tri_words[key] = trigram_counts[key] / bigram_counts[prev]
    else:
        prob_tri_words[key] = trigram_counts[key] / unigram_counts[w2]
        

In [15]:
k = train_n_gram(processed_corpus, 3)

In [18]:
prob_tri_words

{'<s> <s> seriously': 0.0005057034220532319,
 '<s> seriously ask': 0.015037593984962405,
 'seriously ask any': 1.0,
 'ask any woman': 0.06896551724137931,
 'any woman they': 0.4,
 'woman they will': 1.0,
 'they will tell': 0.007743362831858407,
 'will tell you': 0.5172413793103449,
 'tell you this': 0.039473684210526314,
 'you this is': 0.15517241379310345,
 'this is all': 0.011118832522585128,
 'is all very': 0.00784313725490196,
 'all very common': 0.09090909090909091,
 'very common </s>': 0.125,
 'common </s> </s>': 1.0,
 '<s> <s> in': 0.008494296577946769,
 '<s> in us': 0.0040286481647269475,
 'in us of': 0.005780346820809248,
 'us of taxpayers': 0.1111111111111111,
 'of taxpayers paid': 0.5,
 'taxpayers paid tax': 1.0,
 'paid tax </s>': 0.5,
 'tax </s> </s>': 1.0,
 '<s> <s> haha': 0.0010076045627376426,
 '<s> haha russia': 0.0037735849056603774,
 'haha russia go': 1.0,
 'russia go boom': 1.0,
 'go boom splat': 1.0,
 'boom splat india': 1.0,
 'splat india go': 1.0,
 'india go ahhh'

In [17]:
print(k == prob_tri_words)

True


In [ ]:
tri_perplexity = {}
tri_pp = 0
ep=1e-15
for id in test_corpus:
    N = 4 + len(test_corpus[id])
    tokens = ["<s>", "<s>"]
    tokens += test_corpus[id] + ["</s>", "</s>"]
    size = len(tokens)
    temp = 0
    for i in range(0,size-2):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
        if(str in prob_tri_words):
            temp += log(prob_tri_words[str])
        else:
            temp +=log(ep)
    temp = (-temp)/N
    tri_perplexity[id] = exp(temp)
    tri_pp += tri_perplexity[id]

In [ ]:
Trigram_pp = tri_pp/len(test_corpus)
print("Perplexity of Trigram Model without smoothing is: ", Trigram_pp)

Perplexity of Trigram Model without smoothing is:  69139995101.5171


Trigram Model with Laplace Smoothing

In [ ]:
prob_tri_sm_words = {}
for key in trigram_counts:
    w1,w2,w3 = key.split(" ")
    prev = w1 + " " + w2
    if (prev in bigram_counts):
        prob_tri_sm_words[key] = (trigram_counts[key] + 1) / (bigram_counts[prev] + V)
    else:
        prob_tri_sm_words[key] = (trigram_counts[key] + 1) / (unigram_counts[w2] + V)

In [ ]:
tri_perplexity_sm = {}
tri_pp_sm = 0
for id in test_corpus:
    tokens = ["<s>", "<s>"]
    tokens += test_corpus[id] + ["</s>", "</s>"]
    N = len(tokens)
    temp = 0
    for i in range(0,N-2):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
        if(str in prob_tri_sm_words):
            temp += log(prob_tri_sm_words[str])
        else:
            prev = tokens[i] + " " + tokens[i+1]
            if (prev in bigram_counts):
                p = 1 /(bigram_counts[prev] + V)
            elif (tokens[i+1] in unigram_counts):
                p = 1 /(unigram_counts[tokens[i+1]] + V)
            else:
                p = 1/V
            temp += log(p)
    temp = (-temp)/N
    tri_perplexity_sm[id] = exp(temp)
    tri_pp_sm += tri_perplexity_sm[id]

In [ ]:
Trigram_sm_pp = tri_pp_sm/len(test_corpus)
print("Perplexity of Trigram Model is: ", Trigram_sm_pp)

Perplexity of Trigram Model is:  4955.669477563053


Quadgram

In [ ]:
quadgram_counts = {}
for id in processed_corpus:
    tokens = ["<s>", "<s>", "<s>"]
    tokens += processed_corpus[id] + ["</s>", "</s>", "</s>"]
    size = len(tokens)
    for i in range(0,size-3):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2] + " " + tokens[i+3]
        if(str not in quadgram_counts):
            quadgram_counts[str] = 0
        quadgram_counts[str] += 1

In [ ]:
print(len(quadgram_counts))

2910767


In [ ]:
prob_quad_words = {}
for key in quadgram_counts:
    w1,w2,w3,w4 = key.split(" ")
    prev = w1 + " " + w2 + " " + w3
    if (prev in trigram_counts):
        prob_quad_words[key] = quadgram_counts[key] / trigram_counts[prev]
    else:
        prob_quad_words[key] = quadgram_counts[key] / unigram_counts[w3]

In [ ]:
quad_perplexity = {}
quad_pp = 0
ep=1e-15
for id in test_corpus:
    tokens = ["<s>", "<s>","<s>"]
    tokens += test_corpus[id] + ["</s>", "</s>", "</s>"]
    N = len(tokens)
    temp = 0
    for i in range(0,N-3):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2] + " " + tokens[i+3]
        if(str in prob_quad_words):
            temp += log(prob_quad_words[str])
        else:
            temp +=log(ep)
    temp = (-temp)/N
    quad_perplexity[id] = exp(temp)
    quad_pp += quad_perplexity[id]

In [ ]:
Quadgram_pp = quad_pp/len(test_corpus)
print("Perplexity of Quadgram Model without smoothing is: ", Quadgram_pp)

Perplexity of Quadgram Model without smoothing is:  292224555035.2341


Quadgram Model with Laplace Smoothing

In [ ]:
prob_quad_sm_words = {}
for key in quadgram_counts:
    w1,w2,w3,w4 = key.split(" ")
    prev = w1 + " " + w2 + " " + w3
    if (prev in trigram_counts):
        prob_quad_sm_words[key] = (quadgram_counts[key] + 1) / (trigram_counts[prev] + V)
    else:
        prob_quad_sm_words[key] = (quadgram_counts[key] + 1) / (unigram_counts[w3] + V)


In [ ]:
quad_perplexity_sm = {}
quad_pp_sm = 0

for id in test_corpus:
    tokens = ["<s>", "<s>", "<s>"]
    tokens += test_corpus[id] + ["</s>", "</s>", "</s>"]
    N = len(tokens)
    temp = 0
    for i in range(0,N-3):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2] + " " + tokens[i+3]
        if(str in prob_quad_sm_words):
            temp += log(prob_quad_sm_words[str])
        else:
            prev = tokens[i] + " " + tokens[i+1] + " " +tokens[i+2]
            if (prev in trigram_counts):
                p = 1 /(trigram_counts[prev] + V)
            elif ((tokens[i+1]+" " + tokens[i+2]) in bigram_counts):
                p = 1 /(bigram_counts[(tokens[i+1]+" " + tokens[i+2])] + V)
            elif (tokens[i+2] in unigram_counts):
                p = 1/(unigram_counts[tokens[i+2]] + V)
            else:
                p = 1/V
            temp += log(p)
    temp = (-temp)/N
    quad_perplexity_sm[id] = exp(temp)
    quad_pp_sm += quad_perplexity_sm[id]

In [ ]:
Quadgram_sm_pp = quad_pp_sm/len(test_corpus)
print("Perplexity of Quadgram Model with smoothing is: ", Quadgram_sm_pp)

Perplexity of Quadgram Model with smoothing is:  5563.028169800954


Unigram On Train Data

In [ ]:
V_train= len(unigram_counts)

In [ ]:
train_prob_uni_sm_words = {}
for token in unigram_counts:
    train_prob_uni_sm_words[token] = (unigram_counts[token]) / (N_train)

In [ ]:
p=0
for token in unigram_counts:
    p+=log(train_prob_uni_sm_words[token])
p=(-p)/len(unigram_counts)
p=exp(p)
p

1270386.5318923246

In [ ]:
train_uni_perplexity_sm = {}
train_uni_pp_sm = 0
for id in processed_corpus:
    N = 2 + len(processed_corpus[id])
    temp = log(train_prob_uni_sm_words["<s>"])
    for token in processed_corpus[id]:
        temp += log(train_prob_uni_sm_words[token])
    temp += log(train_prob_uni_sm_words["</s>"])
    temp=(-temp)/N
    train_uni_perplexity_sm[id] = exp(temp)
    train_uni_pp_sm += train_uni_perplexity_sm[id]

In [ ]:
train_Unigram_perplexity = train_uni_pp_sm/len(processed_corpus)
print("Perplexity of Unigram Model on train data: ", train_Unigram_perplexity)

Perplexity of Unigram Model on train data:  1236.9767954961758


Bigram on Train Data

In [ ]:
train_prob_bi_sm_words = {}
for key in bigram_counts:
    w1,w2 = key.split(" ")
    train_prob_bi_sm_words[key] = (bigram_counts[key]) / (unigram_counts[w1])

In [ ]:
train_bi_perplexity_sm = {}
train_bi_pp_sm = 0
for id in processed_corpus:
    tokens = ["<s>"]
    tokens += processed_corpus[id]
    tokens.append("</s>")
    N = len(tokens)
    temp = 0
    for i in range(0,N-1):
        str = tokens[i] + " " + tokens[i+1]
        if(str in train_prob_bi_sm_words):
            temp += log(train_prob_bi_sm_words[str])
        elif (tokens[i] in unigram_counts):
            print("bhahhhhh")
            p = 1 / (unigram_counts[tokens[i]] + V_train)
            temp += log(p)
        else:
            print("booooooo")
            p = 1/V_train
            temp += log(p)
    temp = (-temp)/N
    train_bi_perplexity_sm[id] = exp(temp)
    train_bi_pp_sm += train_bi_perplexity_sm[id]

In [ ]:
train_Bigram_perplexity = train_bi_pp_sm/len(processed_corpus)
print("Perplexity of Bigram Model on train data: ", train_Bigram_perplexity)

Perplexity of Bigram Model on train data:  76.8048045805465


Trigram on Train Data

In [ ]:
train_prob_tri_sm_words = {}
for key in trigram_counts:
    w1,w2,w3 = key.split(" ")
    prev = w1 + " " + w2
    if (prev in bigram_counts):
        train_prob_tri_sm_words[key] = (trigram_counts[key]) / (bigram_counts[prev])
    else:
        train_prob_tri_sm_words[key] = (trigram_counts[key]) / (unigram_counts[w2])

In [ ]:
train_tri_perplexity_sm = {}
train_tri_pp_sm = 0
for id in processed_corpus:
    tokens = ["<s>", "<s>"]
    tokens += processed_corpus[id] + ["</s>", "</s>"]
    N = len(tokens)
    temp = 0
    for i in range(0,N-2):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
        if(str in train_prob_tri_sm_words):
            temp += log(train_prob_tri_sm_words[str])
        else:
            prev = tokens[i] + " " + tokens[i+1]
            if (prev in bigram_counts):
                print("A")
                p = 1 /(bigram_counts[prev] + V_train)
            elif (tokens[i+1] in unigram_counts):
                print("B")
                p = 1 /(unigram_counts[tokens[i+1]] + V_train)
            else:
                print("C")
                p = 1/V_train
            temp += log(p)
    temp = (-temp)/N
    train_tri_perplexity_sm[id] = exp(temp)
    train_tri_pp_sm += train_tri_perplexity_sm[id]

In [ ]:
train_tri_pp_sm/len(processed_corpus)

8.646154355511952

Quadgram on train Data

In [ ]:
prob_quad_sm_words = {}
for key in quadgram_counts:
    w1,w2,w3,w4 = key.split(" ")
    prev = w1 + " " + w2 + " " + w3
    if (prev in trigram_counts):
        prob_quad_sm_words[key] = (quadgram_counts[key]) / (trigram_counts[prev])
    else:
        prob_quad_sm_words[key] = (quadgram_counts[key]) / (unigram_counts[w3])


In [ ]:
quad_perplexity_sm = {}
quad_pp_sm = 0

for id in processed_corpus:
    tokens = ["<s>", "<s>", "<s>"]
    tokens += processed_corpus[id] + ["</s>", "</s>", "</s>"]
    N = len(tokens)
    temp = 0
    for i in range(0,N-3):
        str = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2] + " " + tokens[i+3]
        if(str in prob_quad_sm_words):
            temp += log(prob_quad_sm_words[str])
    temp = (-temp)/N
    quad_perplexity_sm[id] = exp(temp)
    quad_pp_sm += quad_perplexity_sm[id]

In [ ]:
Quadgram_sm_pp = quad_pp_sm/len(processed_corpus)
print("Perplexity of Quadgram Model with smoothing is: ", Quadgram_sm_pp)

Perplexity of Quadgram Model with smoothing is:  2.9718372121756267
